# Course on webscraping, morning part

*By Olav ten Bosch and Dick Windmeijer*

In [ ]:
# Imports:
import requests                  # for issueing HTTP requests
from bs4 import BeautifulSoup    # for parsing and navigating HTML results
import time                      # for sleeping between multiple requests
import re                        # for regular expressions

#### Documentation:
- [Requests.py](http://docs.python-requests.org)
- [Beautifulsoup.py](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [regular expressions](https://developers.google.com/edu/python/regular-expressions)

In [ ]:
# Retrieving home page of Statistics Netherlands:
r1 = requests.get('https://www.cbs.nl/en-gb')

print(r1.status_code, r1.headers['content-type'], r1.encoding)
#print(r1.headers)
#print(r1.text)

In [ ]:
# Retrieving home page of Statistics Netherlands with user-agent string:
headers = {'user-agent': 'scrapingCourseBot'}
r2 = requests.get('https://www.cbs.nl/en-gb', headers=headers)

# Headers of the request:
print(r2.request.headers)

# Headers of the response:
print(r2.headers)

In [ ]:
# Issue a request with parameters:
pars = {'products': 2, 'years': 2}
r3 = requests.get('http://testing-ground.scraping.pro/table?', params=pars, headers=headers)        
#print(r3.url)
#print(r3.text)

In [ ]:
# In a loop, always add some idle time (time.sleep) to not overload server:
for products in range(1, 6):
    for years in range(1, 6):
        pars = {'products': products, 'years': years}
        r4 = requests.get('http://testing-ground.scraping.pro/table?', params=pars, headers=headers)
        print(r4.url, r4.status_code)
        time.sleep(1)

In [ ]:
# Use a regexp to retrieve title of main article:
# The main article is in an h2:
#  <h2 style="background-color:#0058b8;" class="bottom">TITLE</h2>
match = re.search(r'<h2 style="background-color:#0058b8;" class="bottom">.*</h2>', r1.text)
if match:
    print(match.group())
else:
    print('not found')

In [ ]:
# Using soup find to access parts of page:
r4 = requests.get('https://www.cbs.nl/en-gb')
soup = BeautifulSoup(r4.text, 'lxml')              # use lxml, is more relaxed in parsing 
print(soup.title.text)
print(soup.find("h2").text)

In [ ]:
# Get the URLS to all news articles of CBS using soup.find:
articles = soup.find_all("div", class_='thumbnail')
for article in articles:
    link = article.find("a")['href']
    print(link)

In [ ]:
headlines = soup.select("div.thumbnail h3")

print(headlines)

In [ ]:
# Get all texts of news articles of CBS using soup.find_all CSS selector:
articles = soup.find_all("div", class_='thumbnail')
links3 = []
for article in articles:
    links3.append(article.find("a")['href'])

for link in links3:
    r = requests.get('https://www.cbs.nl'+link)
    #print(r.url)
    soup2 = BeautifulSoup(r.text, 'lxml')
    leadtext = soup2.find('section', class_='leadtext')
    if leadtext is None: continue
    print(leadtext.text)
    time.sleep(1) # in robots.txt CBS advises a delay of 1 second